<a href="https://colab.research.google.com/github/jiieunx/datacampus/blob/master/%ED%97%88%EA%B7%B8%EB%8D%B0%EC%9D%B4%ED%84%B0_LDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pyLDAvis

In [ ]:
!apt-get update 
!apt-get install g++ openjdk-8-jdk python-dev python3-dev 
!pip3 install JPype1-py3 
!pip3 install konlpy

!JAVA_HOME="C:\Program Files\Java\jdk-14.0.2"


In [ ]:
import re
import numpy as np
import pandas as pd
from pprint import pprint
 
# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
 
# spacy for lemmatization
import spacy
 
# Plotting tools
import pyLDAvis
import pyLDAvis.gensim # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline
 
 
# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)
 
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [ ]:
from collections import Counter
from konlpy.tag import Twitter

In [ ]:
import pandas as pd
df = pd.read_excel('허그일자리.xlsx')
df

In [ ]:
data = df.CONSULT_CONT.values.tolist()

 
# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]
 
# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]
 
pprint(data[:1])

len(data)

In [ ]:
!apt-get update 
!apt-get install g++ openjdk-8-jdk python-dev python3-dev 
!pip3 install JPype1-py3 
!pip3 install konlpy

!JAVA_HOME="C:\Program Files\Java\jdk-14.0.2"
#해야함

In [ ]:
!pip install customized_konlpy
from ckonlpy.tag import Twitter

from collections import Counter
#from konlpy.tag import Twitter

twt = Twitter()
twt.add_dictionary(['허그일자리', '직업훈련', '사후관리', '취업알선','집단상담','경제적','안정적','긍정적','참여자','보호위원','허그일자리사업','구직활동','특이사항','적극적','위문물품','취업박람회','대형면허'], 'Noun') #사용자사전
noun=[]

dataset=[]

for i in range(1350):
  try:

  
    tmp=twt.nouns(data[i])
    stopword=['통해','위해','대한','생으로','고취','다시','참석','맞이','수업','필요','시간','크게','중임','제적','실시','안내','대해','도록','까지','전함','이후','중이','로서','때문','매우','제대로','지금','또힌','자로','지난','오늘','자임','관리','지원','상태','결연','진행','가지','나눔','본인','행사','과정','금일','마련','향후','상황','이야기','위문물품','기반','면서','연락','예정',#의미없는말 
              '허그','일자리','허그일자리사업','허그일자리','멘토링','집단상담','취업','참여','활동','현재','사업','참여자','대상자','보호위원','직업','생활','프로그램','수료식','취업박람회','성공','유선','관련', '모습','설명','훈련','명절','학과','근무','운전면허','입교',#활동이나 행사등 
              '독려', '당부','격려','근속','확인','지속','자녀']# 상담자가 한말    
   # stopword=['통해','위해','대한','생으로','고취','다시','맞이','크게','중임','대해','전함','이후','중이','로서','때문','매우','제대로','지금','또힌','자로','지난','오늘','자임','관리','지원','상태','결연','진행','가지','나눔','본인','행사','과정','금일', #의미없는말 
          #    '허그','일자리','취업','멘토링','상담','참여','활동','현재','집단상담','훈련','사업','참여자','대상자','보호','직업','위원','생활','프로그램','수료식', #활동이나 행사등 
           #   '독려', '당부','격려']# 상담자가 한말    

    #stopword=['통해','위해','대한','생으로','고취','다시','맞이','크게','중임','대해','전함','이후','중이','로서','때문','매우','제대로','지금','또힌','자로','지난','오늘','자임','관리','지원','상태','결연','진행','가지','나눔','본인','행사','과정','금일', #의미없는말 
    #          '허그','일자리','멘토링','활동','현재','사업','참여자','대상자','생활', #활동이나 행사등 
     #         '독려', '당부','격려']# 상담자가 한말  
              #활동 빼면 안될것같다 그 활동에 대한 내용어쨌는지 나와야하니까 넣을 활동과 ㅎ뺄활동 정하기 
    noun += [x for x in tmp if len(x)>1 if x not in stopword]
    dataset.append(noun)
    #noun += [x for x in tmp if len(x)>1]

  except:
    print('except')
            
#java.lang.NullPointerException: java.lang.NullPointerException

In [ ]:
#data_lemmatized=[dataset]
data_lemmatized=dataset

In [ ]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)
 
# Create Corpus
texts =  data_lemmatized
 
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
 
# View
print(corpus[:1])

In [ ]:

id2word[0]

In [ ]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]


In [ ]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
id2word=id2word,
num_topics=10,
random_state=100,
update_every=1,
chunksize=100,
passes=10,
alpha='auto',
per_word_topics=True)

In [ ]:

# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

In [ ]:

 
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus)) # a measure of how good the model is. lower the better.
 
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

In [ ]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis